### create fact table

**READING SILVER DATA**

In [0]:
df_silver = spark.sql("SELECT * FROM PARQUET.`abfss://silver@nvcardatalake.dfs.core.windows.net/carsales`")

**reading all dims**

In [0]:
df_dealer = spark.sql("SELECT * FROM cars_catalog.gold.dim_dealer")
df_branch = spark.sql("SELECT * FROM cars_catalog.gold.dim_branch")
df_date = spark.sql("SELECT * FROM cars_catalog.gold.dim_date")
df_model = spark.sql("SELECT * FROM cars_catalog.gold.dim_model")

**joining - bringing keys to fac table**

In [0]:
df_fact = df_silver.join(df_branch,df_silver['BRANCH_ID'] == df_branch['BRANCH_ID'],how = 'left')\
                    .join(df_date,df_silver['DATE_ID'] == df_date['DATE_ID'],how = 'left')\
                    .join(df_model,df_silver['MODEL_ID'] == df_model['MODEL_ID'],how = 'left')\
                    .join(df_dealer,df_silver['DEALER_ID'] == df_dealer['DEALER_ID'],how = 'left')\
                    .select(df_silver['REVENUE'],df_silver['UNITS_SOLD'],df_silver['RevPerunit'],df_branch['dim_branch_key'],df_date['dim_date_key'],df_model['dim_model_key'],df_dealer['dim_dealer_key'])
df_fact.display()
                    

**writing fact table**

In [0]:
from delta.tables import DeltaTable

In [0]:
if spark.catalog.tableExists('factsales'):
    delta_tbl= DeltaTable.forName(spark,'cars_catlog.gold.factsales')
    delta_tbl.alias('trg').merge(df_fact.alias('src'),'trg.dim_branch_key = src.dim_branch_key and trg.dim_date_key = src.dim_date_key and trg.dim_model_key = src.dim_model_key and trg.dim_dealer_key = src.dim_dealer_key')\
        .whenMatchedUpdateAll()\
        .whenNotMatchedInsertAll()\
        .execute()
else:
    df_fact.write.format('delta')\
        .mode('Overwrite')\
        .option('path','abfss://gold@nvcardatalake.dfs.core.windows.net/factsales')\
        .saveAsTable('cars_catalog.gold.factsales')

In [0]:
%sql
select * from cars_catalog.gold.factsales